In [2]:
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle
import altair as alt
import pytz
import numpy as np
tz = pytz.timezone('America/Edmonton')

In [11]:
def release_token(accessToken):
    path = '/api/ReleaseToken'
    server = 'api.nrgstream.com'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()

with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)

release_token(default_pickle['accessToken'])

In [12]:
t = datetime.now(pytz.timezone('America/Edmonton')).date() - timedelta(days=8)
t2 = datetime.now(pytz.timezone('America/Edmonton')) - timedelta(days=8)
default_pickle = {
    'accessToken':'',
    'current_data':'',
    'daily_outage_dfs':[0]*7,
    'monthly_outage_dfs':[0]*7,
    'alert_dates':{
        'Biomass & Other':t, 
        'Coal':t,
        'Dual Fuel':t,
        'Energy Storage':t,
        'Hydro':t,
        'Natural Gas':t,
        'Solar':t,
        'Wind':t,
        'Intertie':t
    }
}
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Add default current values to dict
with open('./current_df.pickle', 'rb') as handle:
    current_df = pickle.load(handle)
default_pickle['current_data'] = current_df
# Add default monthly values to dict
with open('./monthly_df.pickle', 'rb') as handle:
    monthly = pickle.load(handle)
for i in range(7):
    default_pickle['monthly_outage_dfs'][i] = (t2,monthly)
# Add default daily values to dict
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
for i in range(7):
    default_pickle['daily_outage_dfs'][i] = (t2,daily)
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
default_pickle.keys()

dict_keys(['accessToken', 'current_data', 'daily_outage_dfs', 'monthly_outage_dfs', 'alert_dates'])

In [19]:
default_pickle['alert_dates']

{'Biomass & Other': datetime.date(2022, 9, 19),
 'Coal': datetime.date(2022, 9, 22),
 'Dual Fuel': datetime.date(2022, 9, 4),
 'Energy Storage': datetime.date(2022, 9, 4),
 'Hydro': datetime.date(2022, 9, 4),
 'Natural Gas': datetime.date(2022, 9, 20),
 'Solar': datetime.date(2022, 9, 22),
 'Wind': datetime.date(2022, 9, 16),
 'Intertie': datetime.date(2022, 9, 21)}

In [20]:
with open('./alert_dates.pickle', 'wb') as handle:
    pickle.dump(default_pickle['alert_dates'], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
with open('./daily_df.pickle', 'wb') as handle:
    pickle.dump(default_pickle['daily_outage_dfs'], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
with open('./daily_df.pickle', 'rb') as monthly:
    monthly_df = pickle.load(monthly)
monthly_df[5]

(datetime.datetime(2022, 9, 26, 20, 43, 38, 233194, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                    timeStamp  value              fuelType
 0  2022-09-26 00:00:00-06:00    0.0              Intertie
 1  2022-09-26 01:00:00-06:00    0.0              Intertie
 2  2022-09-26 02:00:00-06:00    0.0              Intertie
 3  2022-09-26 03:00:00-06:00    0.0              Intertie
 4  2022-09-26 04:00:00-06:00    0.0              Intertie
 ..                       ...    ...                   ...
 72 2022-09-29 00:00:00-06:00    0.0  3-Day Solar Forecast
 73 2022-09-29 01:00:00-06:00    0.0  3-Day Solar Forecast
 74 2022-09-29 02:00:00-06:00    0.0  3-Day Solar Forecast
 75 2022-09-29 03:00:00-06:00    0.0  3-Day Solar Forecast
 76 2022-09-29 04:00:00-06:00    0.0  3-Day Solar Forecast
 
 [26829 rows x 3 columns])